# **Topic Modeling**

## **Exercise 1.1: Processing the raw text**

You will using the gensim python library to perform LDA on the Brown corpus

We will start with the file `brown.txt`, which contains 500 documents from the Brown corpus, one document per line, with all punctuation removed.

Read these documents into memory:


In [ ]:
docs = [ line.strip() for line in open('brown.txt', 'r') ]
len(docs)

Split each document into its word tokens and lowercase them, to get a list of lists of words:

In [ ]:
texts = [ [ w for w in d.lower().split() ] for d in docs ]

We could do some more preprocessing, like stoplisting or removing numbers, etc., but you will see that this doesn’t apply for this lab’s general purpose task.

## **Exercise 1.2a: Raw text to bag-of-words**

Many sophisticated models like LDA may rely on the bag-of-words representation (i.e. representing a document as unigram frequencies).

This is done by creating a unique index of all words in the corpus, then transforming each document into a list of those indexes along with a count of how many times they appear.

`gensim` provides a `Dictionary` class for this purpose:

In [ ]:
from gensim.corpora import Dictionary
dictionary = Dictionary(texts)   #Index all words in the corpus
len(dictionary)                  #Number of word types

In [ ]:
dictionary[1729]                 #ID lookup

In [ ]:
dictionary.token2id['rights']    #Word type lookup

In [ ]:
dictionary.token2id['flake']

In [ ]:
allwords = [ dictionary[i] for i in dictionary ]
print(allwords[100:150])

## **Exercise 1.2b: Filtering raw text to bag-of-words**

A common pre-processing task is to remove highly frequent words (e.g. *a, the, of*), as they appear in nearly every document, and thus are uninformative for topics.

For topic modeling, it can also be useful to eliminate words that appear in too few documents, as they are not likely associated with any particular topic.

Both of these tasks can be accomplished using the `.filter_extremes()` function.

Let’s **remove** a word if it appears in fewer than 5 documents (`no_below=5`) or in more than 50%
of the documents (`no_above=0.5`).

In [ ]:
dictionary.filter_extremes(no_below = 5, no_above = 0.5)
len(dictionary) 

In [ ]:
filteredwords = [ dictionary[i] for i in dictionary ]
print(filteredwords[100:150])

Q1. Compare to the above. Which proportion of the vocabular was removed? What do you think was removed?

In [ ]:
dictionary.token2id['rights']        #Indexes can change after filtering 

In [ ]:
#dictionary.token2id['flake']          #Some words have disappeared after filtering 

In [ ]:
#dictionary.token2id['the']            #Some words have disappeared after filtering 

Q2. Check to see if you are right about the some of words that were removed. 

In [ ]:
set(allwords) - set(filteredwords)

Q3. How would you only remove words that occur frequenly across the corpus? Why would you (not) want to do this? (Tip: Review cell 52.)

You can save a dictionary with `.save(filename)`

In [ ]:
dictionary.save('LDALab.bow.dict')

## **Exercise 1.2c: Processing and serializing the corpus**

The function `.doc2bow()` will convert a document to a list of tuples of the unique word indexes in the document and how many times they appear. i.e. [ (word_index, count), (word_index, count), ...]

Do this for the entire collection of texts:

In [ ]:
bow_corpus = [ dictionary.doc2bow(t) for t in texts ]
print (bow_corpus[0])

This can also be done on unseen documents (unseen words will be ignored):

In [ ]:
dictionary.doc2bow('This document has unseen words like Trafalmadorian'.split())

In [ ]:
dictionary[4022], dictionary[7509], dictionary[10999]

Q4. Which words are not indexed? Why?

Serializing such a processed corpus allows us to store it and read it back in another time

In [ ]:
from gensim.corpora import MmCorpus 
MmCorpus.serialize('LDALab.bow.mm', bow_corpus)

## **Exercise 1.3: Bag-of-words to LDA**

Let’s train a **30 topic LDA model** on the first 80% of the documents and evaluate it on the remaining 20%.

Split the bag-of-words corpus into 80% for training and 20% for testing.

In [ ]:
size = int(len(bow_corpus) * 4 / 5)
training = bow_corpus[:size]
testing = bow_corpus[size:]

Now we can train an `LdaModel` object with the following parameters:
  * the processed corpus of interest 
  * `id2word` = the dictionary so that we can print the words instead of the indexes
  * `num_topics` = the number of topics (an important parameter to experiment with)
  * `passes` = the number of iterations for parameter-learning (we only have 500 documents; larger corpora may involve different number of passes for reasonable results)
  
**This might take up to 2 minutes or so to run. Please be patient. You can look ahead at instructions below while waiting.**

In [ ]:
from gensim.models import LdaModel
lda30 = LdaModel(training, id2word=dictionary, num_topics=30, passes=10)

Q5. How do you know if you have chosen a good number of topics?

## **Exercise 1.4a: Evaluating LDA - interpreting**

The `LdaModel` object has a built-in function to print topics/return raw data `.show_topics()`. Its output is hard to read, so a nicer function `pretty_topics()` has been provided for you.

Import the `pretty_topics()` function from `demo` (in the lab directory). It will print the top 10 words and their probabilities per topic. Use this to examine the topics. (Remember: The word distribution is over the full vocabulary, not just frequent words. For a given topic, it sums to 1.)

In [ ]:
from demo import pretty_topics
pretty_topics(lda30)

To see more topics than 2, update the variable num_topics in `pretty_topics()` in demo.py by going to the file,  clicking it to open a text editor, revising the variable value, and then saving the file. Then, restart this notebook with `Kernel` > `Restart & Run All`. Be patient and wait a bit.

Topics will not be exactly the same because of randomness in the model. 

Q6. How difficult/straightforward is it to interpret the topics? Do some make more sense than others? 

Q7. How would you name the topics that do make sense? 


## **Exercise 1.4b: Evaluating LDA - perplexity**

Now we will compute the **perplexity** of the model on the test data.

Remember that LDA is a **generative model**. After training the parameters of the model, let’s test
the likelihood of this model randomly generating the held-out test set.

The `.log_perplexity()` function of the `LdaModel` will give the log perplexity.

In [ ]:
lg_perp = lda30.log_perplexity(testing)
lg_perp

In [ ]:
perp = 2 ** -(lg_perp) # Calculate perplexity 
perp

Your results may not be exactly the same, but should be reasonably close.

For comparison, using a version of this trained model on an external dataset (not given in lab) yielded a perplexity of 42,224 -- much much worse than on our test set from the same corpus.

Q8. For LDA, do you think perplexity is linked to model interpretability?